In [1]:
import argparse
import os
from pathlib import Path

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from accelerate import Accelerator, notebook_launcher
from diffusers import UNet2DModel, DDIMScheduler, DDIMPipeline
from diffusers.optimization import get_cosine_schedule_with_warmup
from diffusers.utils import make_image_grid
from huggingface_hub import create_repo, upload_folder
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [2]:
import os
from PIL import Image
from torch.utils.data import Dataset


class SkyDatasetUnconditional(Dataset):
    def __init__(
        self,
        root_data_folder="/projects/SkyGAN/clouds_fisheye",
        desc_file="processed_1K_JPGs.txt",
        transform=None,
    ):
        self._root_data_folder = root_data_folder
        self._desc_file = desc_file

        self._image_path_list = self._get_image_paths()
        self._n_samples = len(self._image_path_list)

        self._transform = transform

    def _get_image_paths(self):
        file_path = os.path.join(
            self._root_data_folder,
            self._desc_file,
        )
        try:
            with open(file_path) as f:
                paths = f.read().strip().split("\n")
                paths = list(
                    map(
                        lambda x: os.path.join(
                            self._root_data_folder,
                            x,
                        ),
                        paths,
                    )
                )
            return paths
        except FileNotFoundError:
            raise FileNotFoundError(f"Could not find file {self._desc_file} enumerating all image paths")
        except IOError:
            raise IOError(f"An IOError occured while reading file {file_path}. Check correctness of the contents")

    def __getitem__(self, item):
        try:
            sample = Image.open(self._image_path_list[item])
        except FileNotFoundError:
            raise FileNotFoundError(
                f"Could not open file {self._image_path_list[item]}. . Check correctness of the description file {self._desc_file}"
            )

        if self._transform:
            sample = self._transform(sample)

        return sample

    def __len__(self):
        return len(self._image_path_list)


In [3]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 512  # the generated image resolution
    train_batch_size = 4
    eval_batch_size = 16  # how many images to sample during evaluation
    num_epochs = 50
    gradient_accumulation_steps = 1
    learning_rate = 1e-5
    lr_warmup_steps = 500
    save_image_epochs = 3
    save_model_epochs = 3
    mixed_precision = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = "sky_diffusion_ddim_512_lr1e-5_bs4_e50"  # the model name locally and on the HF Hub

    push_to_hub = True  # whether to upload the saved model to the HF Hub
    hub_model_id = "sky_diffusion_ddim_512_lr1e-5_bs4_e50"  # the name of the repository to create on the HF Hub
    hub_private_repo = False
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed = 42


config = TrainingConfig()

In [4]:
def create_model(config: argparse.Namespace) -> UNet2DModel:
    return UNet2DModel(
        sample_size=config.image_size,
        in_channels=3,
        out_channels=3,
        layers_per_block=2,
        block_out_channels=(128, 128, 256, 256, 512, 512),
        down_block_types=(
            "DownBlock2D",
            "DownBlock2D",
            "DownBlock2D",
            "DownBlock2D",
            "AttnDownBlock2D",
            "DownBlock2D",
        ),
        up_block_types=(
            "UpBlock2D",
            "AttnUpBlock2D",
            "UpBlock2D",
            "UpBlock2D",
            "UpBlock2D",
            "UpBlock2D",
        ),
    )

In [5]:
def evaluate(config, epoch, pipeline):
    images = pipeline(
        batch_size=config.eval_batch_size,
        generator=torch.manual_seed(config.seed),
    ).images

    image_grid = make_image_grid(images, rows=4, cols=4)

    test_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{(epoch + 1):04d}.png")

In [6]:
if not os.path.exists(config.output_dir):
    os.makedirs(config.output_dir)
    print(f"Folder {config.output_dir} created")

preprocess = transforms.Compose(
    [
        transforms.Resize((config.image_size, config.image_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)
dataset = SkyDatasetUnconditional(transform=preprocess)
train_dataloader = DataLoader(dataset=dataset, batch_size=config.train_batch_size, shuffle=True)

In [7]:
model = create_model(config)
noise_scheduler = DDIMScheduler()

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)

In [8]:
def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )
    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        if config.push_to_hub:
            print()
            repo_id = create_repo(repo_id=config.hub_model_id or Path(config.output_dir).name, exist_ok=True).repo_id
        accelerator.init_trackers("train_example")

    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0

    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch
            noise = torch.randn(clean_images.shape, device=clean_images.device)
            bs = clean_images.shape[0]

            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device, dtype=torch.int64
            )

            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with accelerator.accumulate(model):
                noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)

                accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        if accelerator.is_main_process:
            pipeline = DDIMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                evaluate(config, epoch + 1, pipeline)

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                torch.save(model.state_dict(), f"{config.output_dir}/models/model{epoch + 1}.pt")
                if config.push_to_hub:
                    upload_folder(
                        repo_id=repo_id,
                        folder_path=config.output_dir,
                        commit_message=f"Epoch {epoch + 1}",
                        ignore_patterns=["step_*", "epoch_*"],
                    )


In [ ]:
args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)
notebook_launcher(train_loop, args, num_processes=1)

Launching training on one GPU.



  0%|          | 0/4710 [00:00<?, ?it/s]

In [ ]:
1